In [1]:
# Constants
DATA_PATH   = "../Data/Processed/3_engineered_df.pkl"

TECH_JOBS = ['Techjobs']

CORE_COLS = ['VersionControlSystem',
             'Languages',
             'Databases',
             'Platforms',
             'WebFrameworks',
             'MiscTech',
             'ToolsTech',
             'CollabTools'
]


MLFLOW_TRACKING_URI = '../models/mlruns'
MLFLOW_EXPERIMENT_NAME = "tech_jobs_predictions"

LOG_PATH = "../models/temp/"
LOG_DATA_PKL    =  "data.pkl"
LOG_MODEL_PKL   =  "model.pkl"
LOG_METRICS_PKL =  "metrics.pkl"

In [2]:
# Load packages
import pandas as pd 
import numpy as np
import logging
import pickle
from pathlib import Path
import os

import mlflow
from mlflow.tracking import MlflowClient

from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import silhouette_score

pd.options.display.max_rows = 1000
pd.options.display.max_columns = 1000

_____

### Functions

_____

In [3]:
# Load dataset and make a copy
eng_df = pd.read_pickle(DATA_PATH)
df = eng_df.copy()

In [4]:
df

Techjobs                 \
      Data scientist or machine learning specialist Engineer, data   
2                                                 1              1   
3                                                 0              0   
9                                                 0              0   
10                                                0              0   
12                                                0              0   
...                                             ...            ...   
73262                                             0              0   
73263                                             0              0   
73264                                             1              0   
73265                                             0              0   
73266                                             0              0   

                                                                           \
      Data or business analyst Developer, back-end Database administrator   
2                            0                   0                      0   
3                            0                   0                      0   
9                            0                   0                      0   
10                           0                   1                      0   
12                           0                   0                      0   
...                        ...                 ...                    ...   
73262                        0                   0                      0   
73263                        0                   1                      0   
73264                        0                   0                      0   
73265                        0                   0                      0   
73266                        0                   0                      0   

                                                                             \
      Developer, mobile Developer, full-stack Cloud infrastructure engineer   
2                     0                     0                             0   
3                     0                     1                             0   
9                     1                     0                             0   
10                    0                     1                             0   
12                    0                     1                             0   
...                 ...                   ...                           ...   
73262                 0                     0                             0   
73263                 0                     0                             0   
73264                 0                     0                             0   
73265                 0                     1                             0   
73266                 0                     0                             0   

                                                                         \
      Developer, embedded applications or devices Developer, QA or test   
2                                               0                     0   
3                                               0                     0   
9                                               0                     0   
10                                              0                     0   
12                                              0                     0   
...                                           ...                   ...   
73262                                           0                     0   
73263                                           0                     0   
73264                                           0                     0   
73265                                           0                     0   
73266                                           0                     0   

                                                            \
      System administrator Scientist Security professional   
2 

### Deal with Imbalanced Dataset

In [5]:
jobs_df =df['Techjobs'].copy()
jobs_df.sum(axis=0).sort_values()

Blockchain                                         736
Security professional                              801
Developer, game or graphics                       1009
Scientist                                         1122
Developer, QA or test                             1425
Data or business analyst                          1666
Database administrator                            1755
Academic researcher                               1819
Engineer, data                                    2091
System administrator                              2099
Developer, embedded applications or devices       2185
Data scientist or machine learning specialist     2268
Cloud infrastructure engineer                     2813
DevOps specialist                                 3350
Developer, mobile                                 4613
Developer, desktop or enterprise applications     5734
Developer, front-end                             10172
Developer, back-end                              19001
Developer,

In [39]:
# DownSample majority classes and OverSample minority Classes
samples_per_class = 1200
resampled_jobs = []

for job in jobs_df.columns:
    sub_df = jobs_df.loc[jobs_df[job] == 1].copy()
    
    if len(sub_df) < samples_per_class:
        # Upsample
        sub_df = sub_df.sample(samples_per_class, replace=True, random_state=42)
    else:
        # Downsample
        sub_df = sub_df.sample(samples_per_class, random_state=42) 
    
    resampled_jobs.append(sub_df)

In [40]:
jobs_df  = pd.concat(resampled_jobs)
jobs_df.sum(axis=0).sort_values()

Blockchain                                       3546
Developer, QA or test                            3671
Developer, game or graphics                      3674
Security professional                            4018
Developer, embedded applications or devices      4070
Developer, front-end                             4193
Engineer, data                                   4201
Scientist                                        4289
Developer, mobile                                4372
Cloud infrastructure engineer                    4395
Database administrator                           4448
Academic researcher                              4544
System administrator                             4580
DevOps specialist                                4610
Data or business analyst                         4652
Data scientist or machine learning specialist    4839
Developer, desktop or enterprise applications    4964
Developer, full-stack                            6392
Developer, back-end         

In [ ]:
len(jobs_df)

22800

### Create a Test Set

### Intialize MLflow

In [11]:
# Create Directories
Path(MLFLOW_TRACKING_URI).mkdir(parents=True, exist_ok=True)
Path(LOG_PATH).mkdir(parents=True, exist_ok=True)

In [12]:
# Initialize client and experiment
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
client = MlflowClient()
mlflow.get_experiment_by_name(MLFLOW_EXPERIMENT_NAME)
exp = client.get_experiment_by_name(MLFLOW_EXPERIMENT_NAME)